Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# 3. Metody tradycyjne

## 3.1. Podział metod tradycyjnych

Pierwsze zaproponowane metody uczenia reprezentacji dla danych grafowych były często metodami **transduktywnymi**. Oznacza to m.in., że raz wyuczony model nie jest w stanie wyznaczyć reprezentacji dla wcześniej nieobserwowanych wierzchołków (ang. *unseen nodes*). Implikuje to również, że cały graf musi być przetworzony przez model – nie można wyuczyć funkcji osadzania (embedding) na części grafu i następnie wyznaczać reprezentacje dla konkretnych węzłów. Modele transduktywne nie są stosowane na dużych (wielkoskalowych) grafach ze względu na wybuchającą liczbę parametrów tych modeli.

Wśród tradycyjnych metod uczenia reprezentacji dla danych grafowych można wyróżnić metody oparte o:
- błądzenia losowe (ang. *random-walks*) – np. DeepWalk, Node2vec,
- faktoryzację macierzy (ang. *matrix factorization*) – np. HOPE, Graph Factorization, GraRep,
- grafowe macierze Laplace'a (ang. *graph Laplacians*) – np. Laplacian Eigenmaps, Local Linear Embedding,
- uczenie głębokie – SDNE, DNGR.

## 3.2. Metody oparte o błądzenia losowe

Omówienie wszystkich metod wykracza poza zakres kursu, ale spróbujemy uruchomić i zbadać dwie wybrane metody oparte o błądzenia losowe, mianowicie **DeepWalk** oraz **Node2vec**. Są to dwie koncepcyjnie bardzo podobne metody. Idea stojąca za tymi metodami jest oparta na modelu word2vec, znanym z dziedziny przetwarzania języka naturalnego. Word2vec wyznacza wektory reprezentacji dla słów w podanym tekście, natomiast w przypadku grafów nie mamy "zdań". Jak zatem zmodyfikować ten algorytm, aby mógł przetwarzać grafy? Na to pytanie po raz pierwszy próbowali odpowiedzieć autorzy metody DeepWalk. Ich rozwiązaniem było próbkowanie grafu za pomocą błądzeń losowych, z których otrzymywali sekwencje wierzchołków. Te sekwencje traktowali jako zdania, w których słowami są wierzchołki. Następnie można było bez problemu zastosować metodę word2vec.

**Uwaga:** Zauważmy, że word2vec nie zwraca uwagi za "treść" słów, a jedynie na *współwystępowanie* słów. Można zatem stosować algorytm word2vec dla dowolnych obiektów o ile można dla nich określić koncepcję współwystępowania.

Pojedyncze błądzenie losowe można opisać za pomocą algorytmu:
1. Rozpocznij od zadanego wierzchołka $u$.
2. Wyznacz zbiór sąsiadów: $\mathcal{N}_u = \{v: (u, v) \in \mathcal{E}\}$.
3. Ze zbioru sąsiadów $\mathcal{N}_u$ wylosuj jeden element $v$ w oparciu o zadany rozkład prawdopodobieństwa.
4. Wróć do kroku 2, ale wyznacz sąsiedztwo wierzchołka $v$. Powtarzaj dopóki nie osiągniesz zadanej długości błądzenia, bądź $\mathcal{N}_u = \emptyset$.

W trakcie przeprowadzania błądzenia losowego zapisywane są kolejno rozważane wierzchołki w postaci sekwencji wierzchołków.


Podsumowując, metoda DeepWalk działa w następujący sposób:
1. Z każdego wierzchołka w grafie rozpocznij błądzenie losowe (z losowaniem sąsiadów opartym na rozkładzie jednostajnym). Wylicz kilka takich błądzeń losowych (hiperparametr metody) o zadanych długościach błądzenia (hiperparametr metody).
2. Zainicjalizuj losowo macierz reprezentacji $\mathbf{Z} \in \mathbb{R}^{|\mathcal{V}| \times d}$.
3. Wyznacz pozytywne pary wierzchołków (współwystępujące w błądzeniach losowych, zastosuj kontekst / okno o zadanej długości – hiperparametr metody).
4. Wylosuj zbiór negatywne pary wierzchołków (liczność zbioru jest hiperparametrem metody).
5. Użyj macierzy reprezentacji oraz pozytywnych i negatywnych par, aby zoptymalizować funkcję kosztu modelu word2vec.

**Uwaga:** Posiadanie (statycznej) macierzy reprezentacji, w której każdy wiersz oznacza reprezentację danego wierzchołka, powoduje, że: (a) metoda nie skaluje się dobrze dla dużych grafów, (b) jest to metoda transduktywna.

Metoda **Node2vec** jest koncepcyjnie bardzo podobna do metody DeepWalk, z taką różnicą, że wprowadzone zostały dwa nowe hiperparametry $p$ oraz $q$, które pozwalają zmieniać zachowanie błądzeń losowych poprzez nadawanie różnych wag sąsiadom danego wierzchołka. Dobierając odpowiednio wartości tych parametrów, błądzenia mogą eksplorować graf na zasadzie przeglądu wszerz (ang. *Breadth-First Search*) albo wgłąb (ang. *Depth-First Search*). Co więcej przymując parametry $p = 1$ oraz $q = 1$ metoda Node2vec jest równoważna z metodą DeepWalk. Stąd też w bibliotece PyTorch-Geometric posiadamy tylko implementację pod nazwą Node2vec.

## 3.3. DeepWalk

Uruchomimy teraz metodą DeepWalk na zbiorze Cora i zwizualizujemy otrzymane wektory reprezentacji.

In [ ]:
from torch_geometric.datasets import Planetoid

data = Planetoid(root="./data/", name="Cora")[0]

In [ ]:
from IPython.display import Code, display


display(Code("random_walk_model.py"))

In [ ]:
from random_walk_model import train_random_walk_model

deepwalk, train_losses = train_random_walk_model(edge_index=data.edge_index, p=1.0, q=1.0)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(range(len(train_losses)), train_losses, linestyle="--", marker="o")
ax.set(xlabel="Epoch", ylabel="Loss")

In [ ]:
import seaborn as sns
from sklearn.decomposition import PCA
from umap import UMAP

from random_walk_model import get_representations


z = get_representations(deepwalk)

z_PCA = PCA(n_components=2).fit_transform(z)
z_UMAP = UMAP(n_components=2).fit_transform(z)

fig, axs = plt.subplots(ncols=2, figsize=(15, 5))
sns.scatterplot(x=z_PCA[:, 0], y=z_PCA[:, 1], hue=data.y, palette="Set2", ax=axs[0])
axs[0].set(title="PCA")
sns.scatterplot(x=z_UMAP[:, 0], y=z_UMAP[:, 1], hue=data.y, palette="Set2", ax=axs[1])
axs[1].set(title="UMAP")

Zaimportujemy funkcje z poprzedniego zeszytu:

In [ ]:
%run "2. Zadania w przetwarzaniu grafów.ipynb"

## 3.4. Ewaluacja w zadaniach

**Klasyfikacja węzłów**

In [ ]:
nc_metrics = evaluate_node_classification(data=data, z=z)

print("-- Node classification --")
print(f"Train AUC: {nc_metrics['train_auc'] * 100.:.2f} [%]")
print(f"Test AUC: {nc_metrics['test_auc'] * 100.:.2f} [%]")

**Predykcja krawędzi**

In [ ]:
lp_dataset = prepare_train_test_sets(edge_index=data.edge_index)

lp_deepwalk, _ = train_random_walk_model(
    edge_index=lp_dataset["train_edges_pos"], 
    p=1.0,
    q=1.0,
    num_epochs=100,
)

lp_z = get_representations(lp_deepwalk)

lp_metrics = evaluate_link_prediction(
    train_edges=lp_dataset["train_edges"],
    y_train=lp_dataset["y_train"],
    test_edges=lp_dataset["test_edges"],
    y_test=lp_dataset["y_test"],
    transformation_name="average",
    z=lp_z,

)

print("-- Link prediction --")
print(f"Train AUC: {lp_metrics['train_auc'] * 100.:.2f} [%]")
print(f"Test AUC: {lp_metrics['test_auc'] * 100.:.2f} [%]")

**Klasyfikacja grafów**

Dla zadania klasyfikacji grafów użyjemy zbioru ENZYMES, w którym każdy graf reprezentuje pojedynczy enzym:

In [ ]:
from torch_geometric.datasets import TUDataset
from tqdm.auto import tqdm


enzymes = TUDataset(root="./data", name="ENZYMES")

print(f"Liczba grafów: {len(enzymes)}")

enzymes_subset, _ = train_test_split(
    [e for e in enzymes], 
    train_size=0.1, 
    stratify=[e.y.item() for e in enzymes],
)
print(f"Ograniczymy liczbę grafów do {len(enzymes_subset)}")

z = []

for enzyme in tqdm(enzymes_subset, desc="Embedding graphs"):
    model, _ = train_random_walk_model(
        edge_index=enzyme.edge_index,
        p=1.0,
        q=1.0,
        num_epochs=10,
        quiet=True,
    )
    
    z.append(get_representations(model))
        
        
y = torch.tensor([e.y for e in enzymes_subset])

gc_metrics = evaluate_graph_classification(z, y, transformation_name="average")

print("-- Graph classification --")
print(f"Train AUC: {gc_metrics['train_auc'] * 100.:.2f} [%]")
print(f"Test AUC: {gc_metrics['test_auc'] * 100.:.2f} [%]")